In [2]:
!pwd

/Users/a8407352/Desktop/urban/final


In [7]:
import pandas as pd
housingDF = pd.read_csv('./UrbanModelling-Segragation/Austin/Data/2014_Housing_Market_Analysis_Data_by_Zip_Code.csv')
housingDF.head()

,Zip Code,Population below poverty level,Median household income,"Non-White, Non-Hispanic or Latino","Hispanic or Latino, of any race",Population with disability,Unemployment,Large households (5+ members),"Homes affordable to people earning less than $50,000","Rentals affordable to people earning less than $25,000",...,Rental units affordable to average teacher,Owner units affordable to average teacher,Rental units affordable to average tech worker,Owner units affordable to average tech worker,"Change in percentage of population below poverty, 2000-2012","Change in median rent, 2000-2012","Change in median home value, 2000-2012",Percentage of homes within 1/4-mi of transit stop,Average monthly transportation cost,Percentage of housing and transportation costs that is transportation-related
0,78726.0,9,66096,16,20,3,4,2,0.0,1,...,70,0.0,99,17.0,375,20,55,2,786,40
1,NaN,19,52431,16,35,8,7,8,24.0,10,...,74,22.0,98,58.0,65,31,74,29,668,38
2,78724.0,38,35711,29,63,8,8,28,85.0,16,...,78,84.0,100,96.0,208,37,46,21,786,40
3,78617.0,18,43957,12,67,10,15,23,NaN,11,...,63,NaN,99,NaN,101,74,21,16,865,42
4,78701.0,20,68152,16,14,10,9,0,7.0,7,...,29,7.0,90,30.0,12,115,59,97,433,23


In [15]:
zipC = housingDF['Zip Code']
from sklearn import linear_model as lm
regr = lm(housingDF['Population below poverty level'])

TypeError: 'module' object is not callable

In [17]:
gentrificationDF = pd.read_csv('./UrbanModelling-Segragation/Austin/Data/Displacement_and_Gentrification_Recommendation_Inventory.csv')
gentrificationDF.head()

,Year (CY),Type,Source,Summary,Recommendation/Resolution,Implementing Department,Action Category,Direct/Indirect?,Targeted at Homelessness?,Actionable?,Status of the Recommendation/Resolution
0,2000,Council Resolution,20000928-037,A resolution authorizing the negotiation and e...,The City Council approves negotiation and exec...,City Manager,Preserving and expanding the supply of afforda...,Direct,No,Actionable,Presumed Implemented
1,2000,Council Resolution,20001130-072,A resolution accepting the Robert Mueller Muni...,The Council accepts the Robert Mueller Municip...,Council,Controlling land for community development,Indirect,No,NaN,Presumed Implemented
2,2000,Council Resolution,20000907-072,A resolution directing the City Manager to ide...,City Council directs the City Manager or his d...,City Manager,Financing,Direct,No,Actionable,"Status unknown: according to CMO, item predate..."
3,2000,Council Resolution,20000302-025,AGREEMENT WITH AUSTIN COMMUNITY DEVELOPMENT CO...,The City Council approves negotiation and exec...,City Manager,Financing,Direct,No,Actionable,Presumed Implemented
4,2000,Council Resolution,20000601-63,DIRECT THE CITY MANAGER TO DEVELOP A COMPREHEN...,The City Manager is directed to address the fe...,City Manager,Controlling land for community development,Indirect,No,NaN,NaN


In [18]:
officersDF = pd.read_csv('./UrbanModelling-Segragation/Austin/Data/gfa7-xe7p.csv')
officersDF.head()

,:@computed_region_8spj_utxs,:@computed_region_a3it_2a2z,:@computed_region_q9nd_rr82,:@computed_region_qwte_z96m,call_type_categories,case,date,day_of_week,hits,less_lethal_force_used_by_apd_prior_to_shooting,location_1,location_1_address,location_1_city,location_1_state,location_1_zip,number_of_officer_shooters,officers_present_when_shots_fired,premise_category,subject_weapon,time
0,NaN,NaN,NaN,NaN,suicidal subject,16-1660189\n16-5024527,2016-06-14T00:00:00.000,Tue,1.0,No,NaN,6000 block Cougar Dr,NaN,NaN,NaN,1,21,Street,knife (machete),6:00 AM
1,8.0,3252.0,2.0,3764.0,shots fired,15-0381788\n15-5005471\n15-5006662,2015-02-08T00:00:00.000,Sun,1.0,No,POINT (-97.888612 30.244609),8206 Pax Dr,NaN,NaN,NaN,1,26,Yard/Courtyard,rifle,12:09 PM
2,NaN,7343.0,NaN,NaN,shots fired,14-3320160\n14-5052475,2014-11-28T00:00:00.000,Fri,1.0,No,POINT (-114.299949 48.192789),715 E 8th St,NaN,NaN,NaN,1,10,Street,rifle,2:22 AM
3,4.0,3646.0,9.0,3851.0,shots fired,13-5048070\n13-5048073\n13-2911294,2013-10-18T00:00:00.000,Fri,1.0,No,POINT (-97.704626 30.379773),1609 Cripple Creek Dr,NaN,NaN,NaN,3,5,Yard/Courtyard,rifle,3:20 PM
4,5.0,3259.0,1.0,NaN,other - nature unknown,17-0531372\n17-5015929\n17-5007713,2017-02-22T00:00:00.000,Wed,2.0,no,POINT (-97.810196 30.1955),7600 MANCHACA RD,NaN,NaN,NaN,1,1,wooded area,knife,6:16 PM


In [23]:
lonLat = officersDF['location_1']
lonLat

0                               NaN
1      POINT (-97.888612 30.244609)
2     POINT (-114.299949 48.192789)
3      POINT (-97.704626 30.379773)
4        POINT (-97.810196 30.1955)
5      POINT (-97.667536 29.881245)
6      POINT (-97.787553 30.443284)
7      POINT (-80.089825 26.489341)
8      POINT (-91.944226 38.848634)
9      POINT (-85.348554 40.178357)
10     POINT (-97.788916 30.167208)
11     POINT (-97.856066 30.181796)
12      POINT (-97.70205 30.379781)
13                              NaN
14     POINT (-97.696992 30.363177)
15      POINT (-97.771937 30.19096)
16     POINT (-97.689696 30.219623)
17     POINT (-97.789496 30.197953)
18      POINT (-92.294899 37.49396)
19      POINT (-97.77643 30.159946)
20      POINT (-98.340468 29.56345)
21     POINT (-97.739649 30.269158)
22      POINT (-97.658896 30.28687)
23      POINT (-97.703314 30.41496)
24     POINT (-97.669664 30.307201)
25     POINT (-97.691009 30.268608)
26     POINT (-97.583783 40.868762)
27     POINT (-97.843473 30.

### Modeling - Hypothesis and Analysis

### Modeling - Assumptions and Limitations